In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sales-prediction/products.csv
/kaggle/input/sales-prediction/bills (1).csv
/kaggle/input/sales-prediction/products.csv
/kaggle/input/sales-prediction/bills (1).csv


In [1]:
!pip install openpyxl xlsxwriter


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 3.2 MB/s eta 0:00:00a 0:00:01


In [8]:
import pandas as pd
from datetime import datetime, timedelta
import ast
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from joblib import Parallel, delayed
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -----------------------------
# 1️⃣ Load data
# -----------------------------
bills_path = "/kaggle/input/sales-prediction/bills (1).csv"
products_path = "/kaggle/input/sales-prediction/products.csv"

bills_df = pd.read_csv(bills_path)
bills_df['Date_Time'] = pd.to_datetime(bills_df['Date_Time'])

products_df = pd.read_csv(products_path)
products_df = products_df.rename(columns={"ID": "ProductID"})
products_df["ProductID"] = products_df["ProductID"].astype(str)

# -----------------------------
# 2️⃣ Helper function to extract items
# -----------------------------
def extract_items(row):
    try:
        product_ids = ast.literal_eval(row['Product_IDs'])
        quantities = ast.literal_eval(row['Quantities'])
        if len(product_ids) != len(quantities):
            return []
        return [{"product_id": str(pid), "quantity_sold": q} for pid, q in zip(product_ids, quantities)]
    except:
        return []

# -----------------------------
# 3️⃣ Summaries: Daily, Weekly, Monthly
# -----------------------------
# Daily
daily_records = []
for _, row in bills_df.iterrows():
    items = extract_items(row)
    for item in items:
        daily_records.append({
            "Date": row['Date_Time'].date(),
            "ProductID": item['product_id'],
            "Quantity_Sold": item['quantity_sold']
        })
daily_sales_df = pd.DataFrame(daily_records)
summary_daily_df = daily_sales_df.groupby(["Date", "ProductID"], as_index=False)["Quantity_Sold"].sum()
summary_daily_df = summary_daily_df.rename(columns={"Quantity_Sold": "Total_Quantity_Sold"})

# Weekly
bills_df['Week_Start'] = bills_df['Date_Time'] - pd.to_timedelta(bills_df['Date_Time'].dt.weekday, unit='d')
weekly_records = []
for _, row in bills_df.iterrows():
    items = extract_items(row)
    week_start = row['Week_Start'].date()
    for item in items:
        weekly_records.append({
            "Week_Start": week_start,
            "ProductID": item['product_id'],
            "Quantity_Sold": item['quantity_sold']
        })
weekly_sales_df = pd.DataFrame(weekly_records)
summary_weekly_df = weekly_sales_df.groupby(["Week_Start", "ProductID"], as_index=False)["Quantity_Sold"].sum()
summary_weekly_df = summary_weekly_df.rename(columns={"Quantity_Sold": "Total_Quantity_Sold"})

# Monthly
bills_df['Month'] = bills_df['Date_Time'].dt.to_period('M')
monthly_records = []
for _, row in bills_df.iterrows():
    items = extract_items(row)
    month_str = str(row['Month'])
    for item in items:
        monthly_records.append({
            "Month": month_str,
            "ProductID": item['product_id'],
            "Quantity_Sold": item['quantity_sold']
        })
monthly_sales_df = pd.DataFrame(monthly_records)
summary_monthly_df = monthly_sales_df.groupby(["Month", "ProductID"], as_index=False)["Quantity_Sold"].sum()
summary_monthly_df = summary_monthly_df.rename(columns={"Quantity_Sold": "Total_Quantity_Sold"})

# -----------------------------
# 4️⃣ Merge with products.csv
# -----------------------------
summary_daily_df["ProductID"] = summary_daily_df["ProductID"].astype(str)
summary_weekly_df["ProductID"] = summary_weekly_df["ProductID"].astype(str)
summary_monthly_df["ProductID"] = summary_monthly_df["ProductID"].astype(str)

summary_daily_df = summary_daily_df.merge(products_df, on="ProductID", how="left")
summary_weekly_df = summary_weekly_df.merge(products_df, on="ProductID", how="left")
summary_monthly_df = summary_monthly_df.merge(products_df, on="ProductID", how="left")

# -----------------------------
# 5️⃣ Brand-level summaries
# -----------------------------
brand_daily = summary_daily_df.groupby(["Date", "Brand Name"], as_index=False)["Total_Quantity_Sold"].sum()
brand_daily.to_csv("brand_daily_sales.csv", index=False)

brand_weekly = summary_weekly_df.groupby(["Week_Start", "Brand Name"], as_index=False)["Total_Quantity_Sold"].sum()
brand_weekly.to_csv("brand_weekly_sales.csv", index=False)

brand_monthly = summary_monthly_df.groupby(["Month", "Brand Name"], as_index=False)["Total_Quantity_Sold"].sum()
brand_monthly.to_csv("brand_monthly_sales.csv", index=False)

# -----------------------------
# 6️⃣ Identify no-sales products
# -----------------------------
all_sold_products = set(summary_daily_df["ProductID"].unique())
no_sales_products = products_df[~products_df["ProductID"].isin(all_sold_products)]
no_sales_products.to_csv("no_sales_products.csv", index=False)



In [10]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from joblib import Parallel, delayed
import numpy as np
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -----------------------------
# Custom Dataset
# -----------------------------
class TimeSeriesDataset(Dataset):
    def __init__(self, series, seq_len):
        self.series = series
        self.seq_len = seq_len

    def __len__(self):
        return len(self.series) - self.seq_len

    def __getitem__(self, idx):
        return (
            torch.tensor(self.series[idx:idx+self.seq_len], dtype=torch.float32).unsqueeze(-1),
            torch.tensor(self.series[idx+self.seq_len], dtype=torch.float32)
        )

# -----------------------------
# GRU with Attention Model
# -----------------------------
class GRUAttention(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=32, num_layers=1):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.attn_fc = nn.Linear(hidden_dim, 1)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        gru_out, _ = self.gru(x)  # (batch, seq_len, hidden_dim)
        attn_weights = torch.softmax(self.attn_fc(gru_out), dim=1)  # (batch, seq_len, 1)
        context_vector = torch.sum(attn_weights * gru_out, dim=1)   # (batch, hidden_dim)
        output = self.fc(context_vector)  # (batch, 1)
        return output

# -----------------------------
# Training + Forecasting Function
# -----------------------------
def train_and_forecast(series, future=30, seq_len=30, epochs=50, batch_size=64):
    scaler = MinMaxScaler()
    series = scaler.fit_transform(series.reshape(-1,1)).flatten()

    dataset = TimeSeriesDataset(series, seq_len)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    model = GRUAttention().to(device)
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Early stopping
    best_loss = float('inf')
    patience = 5
    no_improve = 0

    for epoch in range(epochs):
        epoch_loss = 0
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb).squeeze(-1)
            loss = loss_fn(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(dataloader)
        if avg_loss < best_loss:
            best_loss = avg_loss
            no_improve = 0
        else:
            no_improve += 1

        if no_improve >= patience:
            break

    # Forecast loop
    input_seq = torch.tensor(series[-seq_len:], dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(device)
    preds = []
    model.eval()
    with torch.no_grad():
        for _ in range(future):
            out = model(input_seq).item()
            preds.append(out)
            new_input = torch.cat([input_seq[:,1:,:], torch.tensor([[[out]]], dtype=torch.float32).to(device)], dim=1)
            input_seq = new_input

    preds = scaler.inverse_transform(np.array(preds).reshape(-1,1)).flatten()
    return preds

# -----------------------------
# Forecast per Product
# -----------------------------
def process_product_all(product_id, df, future_daily=30, future_week=7, future_month=30, seq_len=30):
    group = df[df["ProductID"] == product_id].sort_values("Date")
    series = group["Total_Quantity_Sold"].values
    if len(series) < seq_len:
        return None

    daily_forecast = train_and_forecast(series, future=future_daily, seq_len=seq_len)
    week_forecast = train_and_forecast(series, future=future_week, seq_len=seq_len)
    month_forecast = train_and_forecast(series, future=future_month, seq_len=seq_len)

    return {
        "ProductID": product_id,
        "Daily_Forecast": daily_forecast.tolist(),
        "Week_Forecast": week_forecast.tolist(),
        "Month_Forecast": month_forecast.tolist()
    }

# -----------------------------
# Run Forecasting in Parallel
# -----------------------------
product_ids = summary_daily_df["ProductID"].unique()
results = Parallel(n_jobs=-1)(
    delayed(process_product_all)(pid, summary_daily_df) for pid in product_ids
)
results = [r for r in results if r is not None]

forecast_df = pd.DataFrame(results)
forecast_df = forecast_df.merge(products_df[['ProductID', 'Brand Name']], on='ProductID', how='left')
forecast_df.to_csv("product_forecasts_all.csv", index=False)
print("✅ Product-level forecasts saved (daily, weekly, monthly)")

# -----------------------------
# BRAND-LEVEL FORECASTS
# -----------------------------
def aggregate_brand_forecast(forecast_df, col_name, horizon_name):
    expanded = []
    for _, row in forecast_df.iterrows():
        for i, val in enumerate(row[col_name]):
            expanded.append({"Brand": row['Brand Name'], horizon_name: i+1, "Forecast_Qty": val})
    expanded_df = pd.DataFrame(expanded)
    brand_forecast = expanded_df.groupby(["Brand", horizon_name], as_index=False)["Forecast_Qty"].sum()
    return brand_forecast

# Daily
brand_daily_forecast = aggregate_brand_forecast(forecast_df, "Daily_Forecast", "Day")
brand_daily_forecast.to_csv("brand_daily_forecast.csv", index=False)

# Weekly
brand_week_forecast = aggregate_brand_forecast(forecast_df, "Week_Forecast", "Week")
brand_week_forecast.to_csv("brand_week_forecast.csv", index=False)

# Monthly
brand_month_forecast = aggregate_brand_forecast(forecast_df, "Month_Forecast", "Month")
brand_month_forecast.to_csv("brand_month_forecast.csv", index=False)

print("✅ Brand-level forecasts saved (daily, weekly, monthly)")


✅ Product-level forecasts saved (daily, weekly, monthly)
✅ Brand-level forecasts saved (daily, weekly, monthly)


In [4]:
import pandas as pd
import ast
from datetime import datetime
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from joblib import Parallel, delayed
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -----------------------------
# 1️⃣ Load Data
# -----------------------------
bills_path = "/kaggle/input/sales-prediction/bills (1).csv"
products_path = "/kaggle/input/sales-prediction/products.csv"

bills_df = pd.read_csv(bills_path)
products_df = pd.read_csv(products_path)
products_df["ProductID"] = products_df["ID"].astype(str)

# -----------------------------
# 2️⃣ Parse and Explode Data
# -----------------------------
bills_df['Product_IDs'] = bills_df['Product_IDs'].apply(ast.literal_eval)
bills_df['Quantities'] = bills_df['Quantities'].apply(ast.literal_eval)
bills_df['Date'] = pd.to_datetime(bills_df['Date_Time']).dt.date

exploded_df = bills_df.explode(['Product_IDs', 'Quantities'])
exploded_df['ProductID'] = exploded_df['Product_IDs'].astype(str)
exploded_df['Quantity_Sold'] = exploded_df['Quantities'].astype(int)

# -----------------------------
# 3️⃣ Aggregate Daily Sales
# -----------------------------
summary_daily_df = (
    exploded_df.groupby(['Date', 'ProductID'], as_index=False)['Quantity_Sold']
    .sum()
    .rename(columns={'Quantity_Sold': 'Total_Quantity_Sold'})
)

# -----------------------------
# 4️⃣ Identify No-Sales Products
# -----------------------------
all_sold_products = set(summary_daily_df["ProductID"].unique())
no_sales_products_df = products_df[~products_df["ProductID"].isin(all_sold_products)]
no_sales_products_df.to_csv("no_sales_products.csv", index=False)

# Save daily sales summary
summary_daily_df.to_csv("summary_daily_sales.csv", index=False)

# -----------------------------
# 5️⃣ Brand-Level Summaries
# -----------------------------
summary_daily_df = summary_daily_df.merge(products_df[['ProductID', 'Brand Name']], on='ProductID', how='left')

brand_daily = summary_daily_df.groupby(["Date", "Brand Name"], as_index=False)["Total_Quantity_Sold"].sum()
brand_daily.to_csv("brand_daily_sales.csv", index=False)

# -----------------------------
# 6️⃣ Forecasting Model Definition
# -----------------------------
class TimeSeriesDataset(Dataset):
    def __init__(self, series, seq_len):
        self.series = series
        self.seq_len = seq_len

    def __len__(self):
        return len(self.series) - self.seq_len

    def __getitem__(self, idx):
        return (
            torch.tensor(self.series[idx:idx+self.seq_len], dtype=torch.float32).unsqueeze(-1),
            torch.tensor(self.series[idx+self.seq_len], dtype=torch.float32)
        )

class GRUAttention(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=32, num_layers=1):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.attn_fc = nn.Linear(hidden_dim, 1)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        gru_out, _ = self.gru(x)
        attn_weights = torch.softmax(self.attn_fc(gru_out), dim=1)
        context_vector = torch.sum(attn_weights * gru_out, dim=1)
        output = self.fc(context_vector)
        return output

def train_and_forecast(series, future=30, seq_len=30, epochs=50, batch_size=64):
    scaler = MinMaxScaler()
    series = scaler.fit_transform(series.reshape(-1,1)).flatten()

    dataset = TimeSeriesDataset(series, seq_len)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    model = GRUAttention().to(device)
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    best_loss = float('inf')
    patience = 5
    no_improve = 0

    for epoch in range(epochs):
        epoch_loss = 0
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb).squeeze(-1)
            loss = loss_fn(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(dataloader)
        if avg_loss < best_loss:
            best_loss = avg_loss
            no_improve = 0
        else:
            no_improve += 1
        if no_improve >= patience:
            break

    input_seq = torch.tensor(series[-seq_len:], dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(device)
    preds = []
    model.eval()
    with torch.no_grad():
        for _ in range(future):
            out = model(input_seq).item()
            preds.append(out)
            new_input = torch.cat([input_seq[:,1:,:], torch.tensor([[[out]]], dtype=torch.float32).to(device)], dim=1)
            input_seq = new_input

    preds = scaler.inverse_transform(np.array(preds).reshape(-1,1)).flatten()
    return preds

# -----------------------------
# 7️⃣ Forecast Per Product
# -----------------------------
def process_product_all(product_id, df, future_daily=30, future_week=7, future_month=30, seq_len=30):
    group = df[df["ProductID"] == product_id].sort_values("Date")
    series = group["Total_Quantity_Sold"].values
    if len(series) < seq_len:
        return None

    daily_forecast = train_and_forecast(series, future=future_daily, seq_len=seq_len)
    week_forecast = train_and_forecast(series, future=future_week, seq_len=seq_len)
    month_forecast = train_and_forecast(series, future=future_month, seq_len=seq_len)

    return {
        "ProductID": product_id,
        "Daily_Forecast": daily_forecast.tolist(),
        "Week_Forecast": week_forecast.tolist(),
        "Month_Forecast": month_forecast.tolist()
    }

product_ids = summary_daily_df["ProductID"].unique()
results = Parallel(n_jobs=-1)(
    delayed(process_product_all)(pid, summary_daily_df) for pid in product_ids
)
results = [r for r in results if r is not None]

forecast_df = pd.DataFrame(results)
forecast_df = forecast_df.merge(products_df[['ProductID', 'Brand Name']], on='ProductID', how='left')
forecast_df.to_csv("product_forecasts_all.csv", index=False)

# -----------------------------
# 8️⃣ Aggregate Brand Forecasts
# -----------------------------
def aggregate_brand_forecast(forecast_df, col_name, horizon_name):
    expanded = []
    for _, row in forecast_df.iterrows():
        for i, val in enumerate(row[col_name]):
            expanded.append({"Brand": row['Brand Name'], horizon_name: i+1, "Forecast_Qty": val})
    expanded_df = pd.DataFrame(expanded)
    brand_forecast = expanded_df.groupby(["Brand", horizon_name], as_index=False)["Forecast_Qty"].sum()
    return brand_forecast

brand_daily_forecast = aggregate_brand_forecast(forecast_df, "Daily_Forecast", "Day")
brand_daily_forecast.to_csv("brand_daily_forecast.csv", index=False)

brand_week_forecast = aggregate_brand_forecast(forecast_df, "Week_Forecast", "Week")
brand_week_forecast.to_csv("brand_week_forecast.csv", index=False)

brand_month_forecast = aggregate_brand_forecast(forecast_df, "Month_Forecast", "Month")
brand_month_forecast.to_csv("brand_month_forecast.csv", index=False)

print("✅ All steps completed successfully.")

✅ All steps completed successfully.


In [36]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from joblib import Parallel, delayed
import ast

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -----------------------------
# Load Data
# -----------------------------
bills_df = pd.read_csv("/kaggle/input/sales-prediction/bills (1).csv")
products_df = pd.read_csv("/kaggle/input/sales-prediction/products.csv")

# -----------------------------
# Safe Preprocessing & Merge
# -----------------------------
bills_df.columns = bills_df.columns.str.strip()

# Expand multiple Product IDs
bills_df["Product_IDs"] = bills_df["Product_IDs"].astype(str)
bills_df = bills_df.assign(ProductID=bills_df["Product_IDs"].str.split(",")).explode("ProductID")
bills_df["ProductID"] = bills_df["ProductID"].str.strip()

# Convert stringified lists to Python lists for quantities
bills_df["Product_IDs"] = bills_df["Product_IDs"].apply(ast.literal_eval)
bills_df["Quantities"] = bills_df["Quantities"].apply(ast.literal_eval)

# Explode each product and quantity
bills_df = bills_df.explode(["Product_IDs", "Quantities"])
bills_df["ProductID"] = bills_df["Product_IDs"].astype(str).str.strip()
bills_df["Quantity_Sold"] = bills_df["Quantities"].astype(int)

# -----------------------------
# Ensure products_df columns
# -----------------------------
products_df.rename(columns={
    'Product Code': 'ProductID',
    'Product Name': 'Product_Name',
    'Brand Name': 'Brand Name',
    'Price (INR)': 'Price'
}, inplace=True)

products_df["ProductID"] = products_df["ProductID"].astype(str).str.strip()

# Merge bills with products
merged_df = bills_df.merge(products_df[['ProductID', 'Product_Name', 'Brand Name']], on="ProductID", how="left")
merged_df["Date"] = pd.to_datetime(merged_df["Date_Time"]).dt.date

print("✅ Merged shape:", merged_df.shape)

# -----------------------------
# Prepare Daily Summary
# -----------------------------
summary_daily_df = (
    merged_df.groupby(["Date", "ProductID", "Brand Name"])["Quantity_Sold"]
    .sum()
    .reset_index()
    .rename(columns={"Quantity_Sold": "Total_Quantity_Sold"})
)

# -----------------------------
# TimeSeries Dataset
# -----------------------------
class TimeSeriesDataset(Dataset):
    def __init__(self, series, seq_len):
        series = np.array(series, dtype=float)
        if len(series) <= seq_len:
            series = np.pad(series, (0, seq_len + 1 - len(series)), mode="constant", constant_values=0.1)
        self.series = series
        self.seq_len = seq_len

    def __len__(self):
        return max(len(self.series) - self.seq_len, 1)

    def __getitem__(self, idx):
        idx = min(idx, len(self.series) - self.seq_len - 1)
        return (
            torch.tensor(self.series[idx:idx+self.seq_len], dtype=torch.float32).unsqueeze(-1),
            torch.tensor(self.series[idx+self.seq_len], dtype=torch.float32)
        )

# -----------------------------
# GRU + Attention Model
# -----------------------------
class GRUAttention(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=32, num_layers=1):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.attn_fc = nn.Linear(hidden_dim, 1)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        gru_out, _ = self.gru(x)
        attn_weights = torch.softmax(self.attn_fc(gru_out), dim=1)
        context_vector = torch.sum(attn_weights * gru_out, dim=1)
        output = self.fc(context_vector)
        return output

# -----------------------------
# Train and Forecast
# -----------------------------
def train_and_forecast(series, future=30, seq_len=30, epochs=50, batch_size=64):
    if np.sum(series) == 0:
        return np.zeros(future)

    scaler = MinMaxScaler()
    series_scaled = scaler.fit_transform(series.reshape(-1,1)).flatten()

    dataset = TimeSeriesDataset(series_scaled, seq_len)
    batch_size = min(batch_size, len(dataset))
    shuffle = len(dataset) > 1
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

    model = GRUAttention().to(device)
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    best_loss = float('inf')
    patience, no_improve = 5, 0

    for epoch in range(epochs):
        epoch_loss = 0
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb).squeeze(-1)
            loss = loss_fn(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        avg_loss = epoch_loss / len(dataloader)
        if avg_loss < best_loss:
            best_loss = avg_loss
            no_improve = 0
        else:
            no_improve += 1
        if no_improve >= patience:
            break

    # Forecast
    model.eval()
    input_seq = torch.tensor(series_scaled[-seq_len:], dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(device)
    preds = []
    with torch.no_grad():
        for _ in range(future):
            out = model(input_seq).item()
            preds.append(out)
            new_input = torch.cat([input_seq[:,1:,:], torch.tensor([[[out]]], dtype=torch.float32).to(device)], dim=1)
            input_seq = new_input

    preds = scaler.inverse_transform(np.array(preds).reshape(-1,1)).flatten()
    preds = np.maximum(preds, 0)
    return np.round(preds)

# -----------------------------
# Product-level Forecasts
# -----------------------------
def process_product_all(product_id, df, seq_len=30):
    group = df[df["ProductID"] == product_id].sort_values("Date")
    series = group["Total_Quantity_Sold"].values
    if len(series) < 2:
        return None
    return {
        "ProductID": product_id,
        "Brand Name": group["Brand Name"].iloc[0],
        "Daily_Forecast": train_and_forecast(series, future=30, seq_len=seq_len).tolist(),
        "Weekly_Forecast": train_and_forecast(series, future=7, seq_len=seq_len).tolist(),
        "Monthly_Forecast": train_and_forecast(series, future=30, seq_len=seq_len).tolist()
    }

product_ids = summary_daily_df["ProductID"].unique()
results = Parallel(n_jobs=-1)(
    delayed(process_product_all)(pid, summary_daily_df) for pid in product_ids
)
results = [r for r in results if r is not None]

forecast_df = pd.DataFrame(results)
forecast_df.to_csv("product_forecasts_all.csv", index=False)
print("✅ Product-level forecasts saved")

# -----------------------------
# Brand-level Forecasts (aggregate across products)
# -----------------------------
def aggregate_brand_forecast(forecast_df, col_name, horizon_name):
    expanded = []
    for _, row in forecast_df.iterrows():
        brand_col = row.get('Brand Name', 'Unknown')
        for i, val in enumerate(row[col_name]):
            expanded.append({"Brand": brand_col, horizon_name: i+1, "Forecast_Qty": val})
    expanded_df = pd.DataFrame(expanded)
    brand_forecast = expanded_df.groupby(["Brand", horizon_name], as_index=False)["Forecast_Qty"].sum()
    brand_forecast["Forecast_Qty"] = brand_forecast["Forecast_Qty"].round()
    return brand_forecast

brand_daily_forecast = aggregate_brand_forecast(forecast_df, "Daily_Forecast", "Day")
brand_weekly_forecast = aggregate_brand_forecast(forecast_df, "Weekly_Forecast", "Week")
brand_monthly_forecast = aggregate_brand_forecast(forecast_df, "Monthly_Forecast", "Month")

brand_daily_forecast.to_csv("brand_daily_forecast.csv", index=False)
brand_weekly_forecast.to_csv("brand_weekly_forecast.csv", index=False)
brand_monthly_forecast.to_csv("brand_monthly_forecast.csv", index=False)
print("✅ Brand-level forecasts saved (aggregated across products)")


✅ Merged shape: (5398837, 16)
✅ Product-level forecasts saved


KeyError: 'Brand'

In [9]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

# -------------------------
# 1. Load Data
# -------------------------
bills_df = pd.read_csv("/kaggle/input/trail-bills/10000.csv")
products_df = pd.read_csv("/kaggle/input/sales-prediction/products.csv")

# Clean column names
bills_df.columns = bills_df.columns.str.strip()
products_df.columns = products_df.columns.str.strip()

# Force rename the last column in products_df → "ProductID"
products_df.rename(columns={products_df.columns[-1]: "ProductID"}, inplace=True)

# -------------------------
# Process Bills Data
# -------------------------
# Expand multiple product IDs in one bill
bills_df["Product_IDs"] = bills_df["Product_IDs"].astype(str)
bills_df = bills_df.assign(ProductID=bills_df["Product_IDs"].str.split(",")).explode("ProductID")
bills_df["ProductID"] = bills_df["ProductID"].str.strip()

# -------------------------
# Merge with Products
# -------------------------
products_df["ProductID"] = products_df["ProductID"].astype(str).str.strip()
merged_df = bills_df.merge(products_df, on="ProductID", how="left")
merged_df["Date"] = pd.to_datetime(merged_df["Date_Time"]).dt.date

print("✅ Merged shape:", merged_df.shape)
print(merged_df.head())

# -------------------------
# 3. Aggregate Daily Sales
# -------------------------
daily_sales = (
    merged_df.groupby(["Date", "ProductID", "Product Name", "Brand Name"])
    .agg(Total_Sales=("Total_Amount", "sum"))
    .reset_index()
)

# -------------------------
# 4. PyTorch Dataset
# -------------------------
class SalesDataset(Dataset):
    def __init__(self, series, seq_len=7):
        self.scaler = MinMaxScaler()
        self.series = self.scaler.fit_transform(series.reshape(-1, 1))
        self.seq_len = seq_len

    def __len__(self):
        return len(self.series) - self.seq_len

    def __getitem__(self, idx):
        X = self.series[idx:idx+self.seq_len]
        y = self.series[idx+self.seq_len]
        return torch.FloatTensor(X), torch.FloatTensor(y)

# -------------------------
# 5. Simple LSTM Model
# -------------------------
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

# -------------------------
# 6. Train & Forecast
# -------------------------
def train_and_forecast(series, days=30, seq_len=7, epochs=20):
    if len(series) < seq_len + 1:
        return [0] * days  # not enough data

    dataset = SalesDataset(series, seq_len)
    loader = DataLoader(dataset, batch_size=16, shuffle=True)

    model = LSTMModel()
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    # Training
    for epoch in range(epochs):
        for X, y in loader:
            X = X.unsqueeze(-1)  # [batch, seq_len, 1]
            y = y.unsqueeze(-1)
            optimizer.zero_grad()
            output = model(X)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()

    # Forecasting
    scaler = dataset.scaler
    last_seq = dataset.series[-seq_len:]
    preds = []

    for _ in range(days):
        inp = torch.FloatTensor(last_seq).unsqueeze(0).unsqueeze(-1)
        pred = model(inp).item()
        preds.append(pred)
        last_seq = np.append(last_seq[1:], pred)

    preds = scaler.inverse_transform(np.array(preds).reshape(-1, 1)).flatten()
    return preds.tolist()

# -------------------------
# 7. Forecast Products
# -------------------------
product_forecasts = []
for pid, group in daily_sales.groupby("ProductID"):
    series = group.sort_values("Date")["Total_Sales"].values
    preds = train_and_forecast(series)

    weekly_avg = round(np.mean(preds[:7]), 2)
    monthly_avg = round(np.mean(preds[:30]), 2)

    product_forecasts.append({
        "ProductID": pid,
        "Product Name": group["Product Name"].iloc[0],
        "Predictions": preds,
        "Weekly_Avg": weekly_avg,
        "Monthly_Avg": monthly_avg
    })

product_forecasts_df = pd.DataFrame(product_forecasts)
product_forecasts_df.to_csv("product_forecasts.csv", index=False)

# -------------------------
# 8. Forecast Brands
# -------------------------
brand_forecasts = []
for (pid, brand), group in daily_sales.groupby(["ProductID", "Brand Name"]):
    series = group.sort_values("Date")["Total_Sales"].values
    preds = train_and_forecast(series)

    weekly_avg = round(np.mean(preds[:7]), 2)
    monthly_avg = round(np.mean(preds[:30]), 2)

    brand_forecasts.append({
        "ProductID": pid,
        "Brand Name": brand,
        "Predictions": preds,
        "Weekly_Avg": weekly_avg,
        "Monthly_Avg": monthly_avg
    })

brand_forecasts_df = pd.DataFrame(brand_forecasts)
brand_forecasts_df.to_csv("brand_forecasts.csv", index=False)

print("✅ Forecasting complete. Results saved to product_forecasts.csv and brand_forecasts.csv")


✅ Merged shape: (39704, 22)
     Bill_ID  Mobile_Number            Date_Time  \
0  BILL00001     9503299454  2024-07-06 01:09:51   
1  BILL00001     9503299454  2024-07-06 01:09:51   
2  BILL00001     9503299454  2024-07-06 01:09:51   
3  BILL00002     9518832283  2022-11-25 05:51:28   
4  BILL00002     9518832283  2022-11-25 05:51:28   

                                         Product_IDs       Quantities  \
0  ['PVSL0504031125', 'PVSL0502021025', 'PVSL0516...        [9, 2, 4]   
1  ['PVSL0504031125', 'PVSL0502021025', 'PVSL0516...        [9, 2, 4]   
2  ['PVSL0504031125', 'PVSL0502021025', 'PVSL0516...        [9, 2, 4]   
3  ['PVSL0905031125', 'PVSL0306030126', 'PVSL0114...  [5, 5, 3, 5, 3]   
4  ['PVSL0905031125', 'PVSL0306030126', 'PVSL0114...  [5, 5, 3, 5, 3]   

                         Original_Prices                    Offers_Applied  \
0         [36319.38, 19292.36, 35040.03]              ['Yes', 'No', 'Yes']   
1         [36319.38, 19292.36, 35040.03]              ['Yes', 'N

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [42]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from joblib import Parallel, delayed
import ast

# Use a GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----------------------------
# 1. Load Data
# ----------------------------
# Load the bills and products datasets
bills_df = pd.read_csv("/kaggle/input/sales-prediction/bills (1).csv")
products_df = pd.read_csv("/kaggle/input/sales-prediction/products.csv")

# ----------------------------
# 2. Preprocess Bills Data
# ----------------------------
# Clean column names by stripping whitespace
bills_df.columns = bills_df.columns.str.strip()

# Convert Product_IDs and Quantities from string representations to lists
bills_df["Product_IDs"] = bills_df["Product_IDs"].astype(str)
bills_df["Quantities"] = bills_df["Quantities"].apply(ast.literal_eval)

# Explode the DataFrame to have one row per product in each bill
bills_df = bills_df.assign(ProductID=bills_df["Product_IDs"].str.split(",")).explode("ProductID")
bills_df = bills_df.explode(["Quantities"])

# Clean up ProductID and set data types
bills_df["ProductID"] = bills_df["ProductID"].astype(str).str.strip()
bills_df["Quantity_Sold"] = bills_df["Quantities"].astype(int)
bills_df["Date"] = pd.to_datetime(bills_df["Date_Time"]).dt.date

# ----------------------------
# 3. Preprocess Products Data
# ----------------------------
# Rename columns for consistency
products_df.rename(columns={
    'Product Code': 'ProductID',
    'Product Name': 'Product_Name',
    'Brand Name': 'Brand Name',
    'Price (INR)': 'Price'
}, inplace=True)
products_df["ProductID"] = products_df["ProductID"].astype(str).str.strip()

# ----------------------------
# 4. Merge DataFrames
# ----------------------------
# Merge the bills and products dataframes
merged_df = bills_df.merge(products_df[['ProductID', 'Product_Name', 'Brand Name']], on='ProductID', how='left')
# Fill any missing brand names with 'Unknown'
merged_df['Brand Name'] = merged_df['Brand Name'].fillna('Unknown')

# ----------------------------
# 5. Create Daily Sales Summary
# ----------------------------
# Group by Date, ProductID, and Brand Name to get total quantity sold each day
summary_daily_df = (
    merged_df.groupby(["Date", "ProductID", "Brand Name"])["Quantity_Sold"]
    .sum()
    .reset_index()
    .rename(columns={"Quantity_Sold": "Total_Quantity_Sold"})
)

# ----------------------------
# 6. PyTorch Dataset for Time Series
# ----------------------------
class TimeSeriesDataset(Dataset):
    def __init__(self, series, seq_len):
        series = np.array(series, dtype=float)
        # Pad the series if it's shorter than the sequence length
        if len(series) <= seq_len:
            series = np.pad(series, (0, seq_len + 1 - len(series)), 'constant', constant_values=0.1)
        self.series = series
        self.seq_len = seq_len

    def __len__(self):
        return max(len(self.series) - self.seq_len, 1)

    def __getitem__(self, idx):
        # Ensure the index is within bounds
        idx = min(idx, len(self.series) - self.seq_len - 1)
        return (
            torch.tensor(self.series[idx:idx+self.seq_len], dtype=torch.float32).unsqueeze(-1),
            torch.tensor(self.series[idx+self.seq_len], dtype=torch.float32)
        )

# ----------------------------
# 7. GRU with Attention Model
# ----------------------------
class GRUAttention(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=32, num_layers=1):
        super().__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.attn_fc = nn.Linear(hidden_dim, 1)
        self.fc = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        gru_out, _ = self.gru(x)
        attn_weights = torch.softmax(self.attn_fc(gru_out), dim=1)
        context_vector = torch.sum(attn_weights * gru_out, dim=1)
        output = self.fc(context_vector)
        return output

# ----------------------------
# 8. Training and Forecasting Function
# ----------------------------
def train_and_forecast(series, future=30, seq_len=30, epochs=50, batch_size=64):
    # If there's no sales data, predict zeros
    if np.sum(series) == 0 or len(series) == 0:
        return np.zeros(future).tolist()

    # Scale the data
    scaler = MinMaxScaler()
    series_scaled = scaler.fit_transform(series.reshape(-1, 1)).flatten()

    # Create dataset and dataloader
    dataset = TimeSeriesDataset(series_scaled, seq_len)
    batch_size = min(batch_size, len(dataset))
    shuffle = len(dataset) > 1
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

    # Initialize model, loss function, and optimizer
    model = GRUAttention().to(device)
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Training loop with early stopping
    best_loss = float('inf')
    patience, no_improve = 5, 0
    for epoch in range(epochs):
        epoch_loss = 0
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb).squeeze(-1)
            loss = loss_fn(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(dataloader)
        if avg_loss < best_loss:
            best_loss = avg_loss
            no_improve = 0
        else:
            no_improve += 1
        if no_improve >= patience:
            break

    # Forecasting
    model.eval()
    input_seq = torch.tensor(series_scaled[-seq_len:], dtype=torch.float32).unsqueeze(0).unsqueeze(-1).to(device)
    preds = []
    with torch.no_grad():
        for _ in range(future):
            out = model(input_seq).item()
            preds.append(out)
            new_input = torch.cat([input_seq[:, 1:, :], torch.tensor([[[out]]], dtype=torch.float32).to(device)], dim=1)
            input_seq = new_input

    # Inverse transform the predictions to get the actual scale
    preds = scaler.inverse_transform(np.array(preds).reshape(-1, 1)).flatten()
    preds = np.maximum(preds, 0)
    return np.round(preds).tolist()

# ----------------------------
# 9. Generate Forecasts per Product
# ----------------------------
def process_product_all(product_id, df, seq_len=30):
    group = df[df["ProductID"] == product_id].sort_values("Date")
    series = group["Total_Quantity_Sold"].values
    # Pad the series if it's too short
    if len(series) < 2:
        series = np.pad(series, (0, 2 - len(series)), 'constant', constant_values=0.1)

    # Generate daily, weekly, and monthly forecasts
    return {
        "ProductID": product_id,
        "Brand Name": group["Brand Name"].iloc[0],
        "Daily_Forecast": train_and_forecast(series, future=30, seq_len=seq_len),
        "Weekly_Forecast": train_and_forecast(series, future=7, seq_len=seq_len),
        "Monthly_Forecast": train_and_forecast(series, future=30, seq_len=seq_len)
    }

# Run the forecasting in parallel for speed
product_ids = summary_daily_df["ProductID"].unique()
results = Parallel(n_jobs=-1)(delayed(process_product_all)(pid, summary_daily_df) for pid in product_ids)
results = [r for r in results if r is not None]

# Save the product-level forecasts
forecast_df = pd.DataFrame(results)
forecast_df.to_csv("product_forecasts_all.csv", index=False)
print("✅ Product-level forecasts saved to product_forecasts_all.csv")

# ----------------------------
# 10. Aggregate Forecasts by Brand
# ----------------------------
def aggregate_brand_forecast(forecast_df, col_name, horizon_name):
    expanded = []
    for _, row in forecast_df.iterrows():
        # Handle cases where brand name might be null
        brand_col = row['Brand Name'] if pd.notnull(row['Brand Name']) else 'Unknown'
        for i, val in enumerate(row[col_name]):
            expanded.append({"Brand": brand_col, horizon_name: i + 1, "Forecast_Qty": val})

    expanded_df = pd.DataFrame(expanded)
    # Group by brand and the time horizon (Day, Week, Month)
    brand_forecast = expanded_df.groupby(["Brand", horizon_name], as_index=False)["Forecast_Qty"].sum()
    brand_forecast["Forecast_Qty"] = brand_forecast["Forecast_Qty"].round()
    return brand_forecast

# Generate and save brand-level forecasts
brand_daily_forecast = aggregate_brand_forecast(forecast_df, "Daily_Forecast", "Day")
brand_weekly_forecast = aggregate_brand_forecast(forecast_df, "Weekly_Forecast", "Week")
brand_monthly_forecast = aggregate_brand_forecast(forecast_df, "Monthly_Forecast", "Month")

brand_daily_forecast.to_csv("brand_daily_forecast.csv", index=False)
brand_weekly_forecast.to_csv("brand_weekly_forecast.csv", index=False)
brand_monthly_forecast.to_csv("brand_monthly_forecast.csv", index=False)
print("✅ Brand-level forecasts saved to brand_daily_forecast.csv, brand_weekly_forecast.csv, and brand_monthly_forecast.csv")

✅ Product-level forecasts saved to product_forecasts_all.csv
✅ Brand-level forecasts saved to brand_daily_forecast.csv, brand_weekly_forecast.csv, and brand_monthly_forecast.csv
